In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv("/mnt/d/datasets/puzzles/lichess_db_puzzle.csv")
df = df[["FEN", "Moves", "Rating", "Themes"]]
df = df[df["Rating"] < 3000]
df["Bucket"] = df["Rating"] // 100
df

,FEN,Moves,Rating,Themes,Bucket
0,r6k/pp2r2p/4Rp1Q/3p4/8/1N1P2R1/PqP2bPP/7K b - ...,f2g3 e6e7 b2b1 b3c1 b1c1 h6c1,1995,crushing hangingPiece long middlegame,19
1,5rk1/1p3ppp/pq3b2/8/8/1P1Q1N2/P4PPP/3R2K1 w - ...,d3d6 f8d8 d6d8 f6d8,1452,advantage endgame short,14
2,8/4R3/1p2P3/p4r2/P6p/1P3Pk1/4K3/8 w - - 1 64,e7f7 f5e5 e2f1 e5e6,1276,advantage endgame rookEndgame short,12
3,r2qr1k1/b1p2ppp/pp4n1/P1P1p3/4P1n1/B2P2Pb/3NBP...,b6c5 e2g4 h3g4 d1g4,1099,advantage middlegame short,10
4,8/8/4k1p1/2KpP2p/5PP1/8/8/8 w - - 0 53,g4h5 g6h5 f4f5 e6e5 f5f6 e5f6,1556,crushing endgame long pawnEndgame,15
...,...,...,...,...,...
3764374,3r3k/p5pp/8/5R2/1BQ1p3/P3q3/Bb4PP/6K1 w - - 0 28,g1f1 d8d1 b4e1 e3e1,1025,mate mateIn2 middlegame short,10
3764375,5Q2/pp3R1P/1kpp4/4p3/2P1P3/3PP2P/Pr2q3/2K5 w -...,f7f2 b2c2 c1b1 e2d1,1674,endgame mate mateIn2 queensideAttack short,16
3764376,r3kb1r/ppp2ppp/2n5/3q3b/3P1B2/5N1P/PPP3P1/RN1Q...,c6d4 f1e1 e8d8 b1c3 d4f3 g2f3,2507,advantage long opening,25
3764377,r2q1rk1/4bppp/p1n1pn2/1p1pN3/2pP2b1/1PP1P3/PBQ...,g4f5 e5c6 f5c2 c6d8 c2b1 d8c6,2256,crushing hangingPiece long middlegame,22


In [3]:
from itertools import chain
unique_lists_in_items = df.Themes.str.split(" ").apply(lambda x: frozenset(x)).unique().tolist()
all_themes = set(chain(*unique_lists_in_items))
all_themes

{'advancedPawn',
 'advantage',
 'anastasiaMate',
 'arabianMate',
 'attackingF2F7',
 'attraction',
 'backRankMate',
 'bishopEndgame',
 'bodenMate',
 'capturingDefender',
 'castling',
 'clearance',
 'crushing',
 'defensiveMove',
 'deflection',
 'discoveredAttack',
 'doubleBishopMate',
 'doubleCheck',
 'dovetailMate',
 'enPassant',
 'endgame',
 'equality',
 'exposedKing',
 'fork',
 'hangingPiece',
 'hookMate',
 'interference',
 'intermezzo',
 'kingsideAttack',
 'knightEndgame',
 'long',
 'master',
 'masterVsMaster',
 'mate',
 'mateIn1',
 'mateIn2',
 'mateIn3',
 'mateIn4',
 'mateIn5',
 'middlegame',
 'oneMove',
 'opening',
 'pawnEndgame',
 'pin',
 'promotion',
 'queenEndgame',
 'queenRookEndgame',
 'queensideAttack',
 'quietMove',
 'rookEndgame',
 'sacrifice',
 'short',
 'skewer',
 'smotheredMate',
 'superGM',
 'trappedPiece',
 'underPromotion',
 'veryLong',
 'xRayAttack',
 'zugzwang'}

In [4]:
# settings
X = 1000
downsample = False

In [5]:
# sample at least X puzzles for each theme
rows = []
for theme in tqdm(all_themes):
    df_theme = df[df.Themes.str.contains(theme)]
    rows.append(df_theme.sample(min(X, len(df_theme))))
df_themes = pd.concat(rows)

# sample at least X puzzles for each rating bucket
df_ratings = df.groupby("Bucket").sample(X)

# remove duplicates
df_sampled = pd.concat([df_themes, df_ratings]).drop_duplicates()
df_sampled = df_sampled.sample(frac=1).reset_index(drop=True) # shuffle

100%|██████████| 60/60 [00:23<00:00,  2.57it/s]


In [6]:
df_downsampled = df_sampled.copy()

if downsample:
    # try to reduce the number of rows by removing puzzles and checking the counts
    for i in tqdm(range(df_downsampled.shape[0])):
        # remove index i
        df_without_i = df_downsampled.drop(df_downsampled.iloc[i].name)

        still_ok = True

        # check if the number of unique puzzles is at least 100
        for theme in all_themes:
            if len(df_without_i[df_without_i.Themes.str.contains(theme)]) < 100:
                still_ok = False
                break
        # check if the number of buckets is at least 100
        if df_without_i.groupby("Bucket").count().min()[0] < 100:
            still_ok = False
        
        if still_ok:
            df_downsampled = df_without_i

df_downsampled
# the error below is expected, run the next cell

,FEN,Moves,Rating,Themes,Bucket
0,5r1k/3q1rp1/p4pQN/1p1p4/3P4/4P2R/PP4PP/5nK1 b ...,g7h6 g6h6 f7h7 h6f8,1914,deflection endgame mate mateIn2 short,19
1,2r3k1/r1N2p1p/3P1Qp1/2n1p3/4P3/5B2/6PP/6K1 w -...,f3g4 a7a1 f6f1 a1f1,1097,crushing endgame short,10
2,4r3/5k1p/4p1p1/3pQ1PP/p4P2/5q2/1P5K/4R3 b - - ...,f3h5 h2g3 h7h6 e1c1 h6g5 f4g5 f7g8 c1c7 h5h8 e5f4,2918,crushing defensiveMove endgame quietMove veryLong,29
3,5k2/5p2/p4Np1/R5Kp/5P2/3b2P1/1P4r1/4B1b1 b - -...,f8g7 a5d5 g2b2 d5d3 b2b5 f6d5,2717,advantage defensiveMove endgame long,27
4,r2q1rk1/2pn1p2/p2p2pp/1p1P4/2PB2Q1/bB5P/5PP1/1...,d7c5 b3c2 d8g5 g4f3,1772,advantage middlegame short,17
...,...,...,...,...,...
85693,rn2kb1r/1bq2ppp/p7/3n2B1/1p1N4/5P2/PPP1B1PP/R3...,f8c5 e2b5 e8f8 e1e8,1480,doubleCheck mate mateIn2 opening short,14
85694,r2q1rk1/p1p2ppp/b1p1p3/b3P3/2pPnP2/P1N5/1PQ3PP...,d8h4 g2g3 h4d8 c2e4,2210,defensiveMove equality middlegame short,22
85695,8/1P6/3k4/3Bp1pp/4P3/5PK1/6PP/1r6 b - - 1 49,b1b2 h2h4 g5h4 g3h4 d6c7 g2g4 b2h2 h4g3,1810,crushing endgame veryLong,18
85696,2rr2k1/1p2bpp1/p1N1p2p/3nR3/7B/2PB4/PP3PPP/3R2...,c8c6 h4e7 d5e7 d3h7 g8h7 d1d8,1188,advantage discoveredAttack kingsideAttack long...,11


In [7]:
sub_df = df_downsampled[["FEN", "Moves", "Rating", "Themes"]]
sub_df.to_csv(f"puzzles-{X}.csv", index=False, header=False)